# Initialize Pose

In [43]:
#   [X] receive initial pose with covariance from ROS parameter
#   [X] generate particles around the initial pose uniformly or based on the covariance
#   [X] subscribe to /initialpose topic to get messages from RViz (2D Pose Estimation function) to reset the pose
#   [ ] self.click_sub = rospy.Subscriber("/clicked_point", PointStamped, self.clicked_pose, queue_size=1)

In [15]:
import numpy as np 
num_particles = 100
orientations = np.random.uniform(0, 2 * np.pi, size=num_particles)
print(orientations* (180 / np.pi))

[ 17.87537741 254.35718482 217.98440569 181.81257378 314.13961327
 286.74030257 264.29128326 326.98838343 105.22171798  30.86539112
 202.63249711 340.18755789 198.44679628 291.51032085   2.85544057
 312.04375461 277.53558519  80.73123551 284.64498132 278.51070387
 278.93782308   0.57082697 237.51220277 245.79260961 346.56843646
  34.23799384 157.27073581  15.63187219 252.76330842  22.57396222
  50.00043899 274.08591283 260.41599668 285.0761861  133.13029428
 120.5657396  280.63644646  90.40652743 164.53793561 342.63343847
  68.71232214 244.97633462 178.84829204 297.88715185  73.92426751
 157.6959842  292.06947051  83.30690162 294.69820069 158.53728984
 180.12749972  84.22568489 295.26099736  82.81014101  48.67994555
  28.02493414 119.72444193 300.00098791 337.4864067   99.7527543
 135.99189874 292.43210999 115.85932827  26.89529329 341.10699181
 328.87197421 286.63331383 274.33323324 241.62429667 223.44212476
 135.4697202  297.7077119  315.75333682 300.57034474 350.69144761
 123.007291

In [8]:
import numpy as np 

num_particles = 5
init_pose_x = 0.0

weights = np.ones(num_particles)/num_particles 
particles = np.zeros(shape=(num_particles, 3))
particles[:, 0] = init_pose_x + np.random.normal(scale=0.5, size=num_particles)       
        
print(particles)

[[ 0.19347906  0.          0.        ]
 [ 0.09035276  0.          0.        ]
 [ 0.07821083  0.          0.        ]
 [-0.38466976  0.          0.        ]
 [ 0.25187269  0.          0.        ]]


### initial pose with covariance 

In [25]:
import numpy as np 

num_particles = 1000

# initial pose from ROS parameter
x = 0.0 
y = 0.0 
theta = 0.0 
mean = [x, y, theta]

# initial cov from ROS parameter
covariance = np.array([[0.5, 0.0, 0.0],
                       [0.0, 0.5, 0.0],
                       [0.0, 0.0, 0.1]])

# particle distribution over multivariate normal distribution with mean = init_pose and covariance = init_cov
particles = np.random.multivariate_normal(mean, covariance, size=num_particles)

### global localization

initial pose and initial covariance not given

In [29]:
# ROS parameter
initial_state = False 

# do not use initial state -> global localization 
if not initial_state: 
    
    # particle distribution uniformly over all free cells in the occupancy grid
    import numpy as np

    num_particles = 1000

    # map parameter: we want to use the whole map
    map_width = 1000
    map_height = 200

    # ranges for selecting possible poses 
    x_range = [0, map_width]  
    y_range = [0, map_height]  
    theta_range = [-np.pi, np.pi]  

    # particle distribution uniformly over the map 
    # TODO: use only free cells from the occupancy grid
    particles = np.zeros((num_particles, 3))
    particles[:, 0] = np.random.uniform(x_range[0], x_range[1], size=num_particles)  # x
    particles[:, 1] = np.random.uniform(y_range[0], y_range[1], size=num_particles)  # y
    particles[:, 2] = np.random.uniform(theta_range[0], theta_range[1], size=num_particles)  # theta


### Reset Pose

subscribe to /initialpose topic to get messages from RViz (2D Pose Estimation function) to reset the pose

In [42]:
from geometry_msgs.msg import PoseWithCovarianceStamped
from tf_transformations import euler_from_quaternion
from geometry_msgs.msg import Quaternion

num_particles = 1000

def initial_pose_cb(msg: PoseWithCovarianceStamped):
    # TODO: probably reset some stuff 
    
    # extract position
    x = msg.pose.pose.position.x
    y = msg.pose.pose.position.y 
    
    # extract orientation from quaternium 
    q : Quaternion = msg.pose.pose.orientation
    _, _, theta = euler_from_quaternion(q.x, q.y, q.z, q.w)
    
    
    # mean 
    mean = [x, y, theta]
    
    # covariance constants 
    POSE_COVARIANCE_XX = 0.5    
    POSE_COVARIANCE_YY = 0.5
    POSE_COVARIANCE_AA = 0.1
    
    # covariance
    covariance = np.array([[POSE_COVARIANCE_XX, 0.0, 0.0],
                        [0.0, POSE_COVARIANCE_YY, 0.0],
                        [0.0, 0.0, POSE_COVARIANCE_AA]])
    
    # particle distribution over multivariate normal distribution with mean = init_pose and covariance = init_cov
    particles = np.random.multivariate_normal(mean, covariance, size=num_particles)


In [41]:
# import rclpy 
# from rclpy.node import Node
# from rclpy.qos import QoSProfile, QoSDurabilityPolicy, QoSReliabilityPolicy, QoSHistoryPolicy
# from geometry_msgs.msg import PoseWithCovarianceStamped

# qos_profile = QoSProfile(
#     history=QoSHistoryPolicy.KEEP_LAST,         # Keep only the last message
#     depth=1,                                    # Store only 1 message
#     reliability=QoSReliabilityPolicy.RELIABLE,  # Ensure reliable delivery
#     durability=QoSDurabilityPolicy.VOLATILE     # Messages are not persisted
# )

# # reliability=RELIABLE:
# #     Garantiert die Zustellung der Nachricht.
# #     Wichtig, da die Initial Pose eine kritische Information für das Lokalisierungssystem ist.

# # durability=VOLATILE:
# #     Die Nachricht wird nur an Subscriber gesendet, die aktiv mit dem Publisher verbunden sind, während die Nachricht veröffentlicht wird.
# #     Das ist sinnvoll, da es keine historische Datenhaltung erfordert.


# initial_pose_sub = Node.create_subscription(Node, PoseWithCovarianceStamped, '/initial_pose', initial_pose_cb, qos_profile)

TypeError: Node._validate_qos_or_depth_parameter() missing 1 required positional argument: 'qos_or_depth'